In [1]:
from lib2to3.pytree import WildcardPattern
from xml.dom import minidom
from dataclasses import dataclass
import json
import lxml
from bs4 import BeautifulSoup, SoupStrainer
from xml.dom import minidom
import os
import argparse
import bz2
import logging
import os.path
import re  # TODO use regex when it will be standard
import sys
from io import StringIO
from multiprocessing import Queue, get_context, cpu_count
from timeit import default_timer
import re
import pymorphy2
from functools import lru_cache
from tqdm import tqdm
import pickle
from ruwordnet import RuWordNet
from collections import defaultdict
import time
from  extractor import Extractor
import numpy as np
import bert
from cmT8aq2P import SentenceBertTransformer
import torch
from transformers import AutoModel, AutoTokenizer

0.0025435425796423165


In [2]:
def my_split(x):
    s = ""
    for i in x:
        if i!= "(" and i != ")":
            s +=i
        elif i  == ")":
            s += ""
        else:
            s += ","
    return s
@lru_cache(maxsize=200000)
def get_normal_form(word):
    return morph_analizer.parse(word)[0].normal_form
acceptedNamespaces = ['w', 'wiktionary', 'wikt']
templateNamespace = ''
morph_analizer = pymorphy2.MorphAnalyzer()
tagRE = re.compile(r'(.*?)<(/?\w+)[^>]*>(?:([^<]*)(<.*?>)?)?')
def collect_pages(text):
    """
    :param text: the text of a wikipedia file dump.
    """
    # we collect individual lines, since str.join() is significantly faster
    # than concatenation
    page = []
    id = ''
    revid = ''
    last_id = ''
    inText = False
    redirect = False
    redirect_page = ''
    for line in text:
        if '<' not in line:     # faster than doing re.search()
            if inText:
                page.append(line)
            continue
        m = tagRE.search(line)
        if not m:
            continue
        tag = m.group(2)
        if tag == 'page':
            page = []
            redirect = False
        elif tag == 'id' and not id:
            id = m.group(3)
        elif tag == 'id' and id: # <revision> <id></id> </revision>
            revid = m.group(3)
        elif tag == 'title':
            title = m.group(3)
        elif tag == 'redirect':
            redirect = True
            redirectRE = re.compile(r'title=\"(.*?)\" />')
            redirect_page = re.findall(redirectRE, line)[0]
        elif tag == 'text':
            inText = True
            line = line[m.start(3):m.end(3)]
            page.append(line)
            if m.lastindex == 4:  # open-close
                inText = False
        elif tag == '/text':
            if m.group(1):
                page.append(m.group(1))
            inText = False
        elif inText:
            page.append(line)
        elif tag == '/page':
            colon = title.find(':')
            if (colon < 0 or (title[:colon] in acceptedNamespaces) and id != last_id and
                    not redirect and not title.startswith(templateNamespace)):
                yield (id, revid, title, page, redirect_page,redirect)
                last_id = id
            id = ''
            revid = ''
            page = []
            inText = False
            redirect = False
            redirect_page=''

def decode_open(filename, mode='rt', encoding='utf-8'):
    """
    Open a file, decode and decompress, depending on extension `gz`, or 'bz2`.
    :param filename: the file to open.
    """
    ext = os.path.splitext(filename)[1]
    if ext == '.gz':
        import gzip
        return gzip.open(filename, mode, encoding=encoding)
    elif ext == '.bz2':
        return bz2.open(filename, mode=mode, encoding=encoding)
    else:
        return open(filename, mode, encoding=encoding)
def extract_cat(text):
    matcher=re.compile(r"Категория:\s?([А-Яа-я\s?]+)")
    return matcher.findall(text)
def extract_links(text):
    matcher=re.compile(r"[\[\[]([А-Яа-я\s?]+)[\|,\]\]]")
    return matcher.findall(text)
def extract_first_links(text):
    matcher=re.compile(r"[\[\[]([А-Яа-я\s?]+)[\|,\]\]]")
    answer = []
    for elem in text.split("\n"):
        item = matcher.findall(elem)
        if len(item) > 0:
            answer.append(item[0])
    return answer

In [3]:
@dataclass
class Page:
    id: int
    revid: int
    title:str
    meaningPage: bool
    multiPage: bool
    categories: list
    links: list
    redirect:bool
    first_sentence:str
    def __eq__(self, other):
        return (self.id, self.revid, self.title, self.meaningPage,
                self.multiPage, self.categories, self.links, 
                self.redirect) == (other.id, other.revid, 
                other.title, other.meaningPage,
                other.multiPage, other.categories, other.links, 
                other.redirect)
class PageEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Page):
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)

In [4]:
input = decode_open("D:\\asd\\ruwiki-20220701-pages-meta-current.xml.bz2")
i = 0
dictRedirect = {}
pages = []
redirectcount = 0
dictPageRedirect = {}
# log = open("log.txt", "w", encoding="utf-8")
i = 0
for id, revid, title, page, redirect_page, redirect in tqdm(collect_pages(input)):
    # print(title, "aaaa",''.join(page), file = printt)
    i += 1
    text = ''.join(page)
    text_lower = text.lower()
    multiPage = False
    if text_lower.find('{{другие значения') != -1:
        multiPage = True
    elif title.find("(") != -1 and (not "значения" in title.lower())and (not "значение" in title.lower()):
        multiPage = True
    elif text_lower.find("{{перенаправление") != -1:
        multiPage = True
    elif text_lower.find("{{другое значение") != -1:
        multiPage = True
    elif text_lower.find("{{значения") != -1:
        multiPage = True
    elif text_lower.find("{{redirect-multi") != -1:
        multiPage = True
    elif text_lower.find("{{redirect-multi") != -1:
        multiPage = True
    elif text_lower.find("{{see also") != -1:
        multiPage = True
    elif text_lower.find("{{о|") != -1:
        multiPage= True
    elif text_lower.find("{{список однофамильцев}}") != -1:
        multiPage= True
    categories = extract_cat(text)
    
    meaningPage = False
    if ("значения" in title.lower()) or ("значение" in title.lower()):
        meaningPage = True
    elif text_lower.find('{{неоднозначность') != -1:
        meaningPage = True
    elif text_lower.find('{{многозначность') != -1:
        meaningPage = True
    elif text_lower.find('{{disambig') != -1:
        meaningPage = True
    # redirects = extract_redirects(text)
    links =[]
    if not meaningPage:
        links = extract_links(text)
    else:
        links = extract_first_links(text)
    first_sentense = ""
    if not redirect_page:
        ext = Extractor(id,revid,"",title,page)
        first_sentense = "\n".join(ext.clean_text(text)).split(".")[0]
        
    if len(redirect_page) > 0:
        if redirect_page not in dictRedirect:
            dictRedirect[redirect_page] = []
            dictPageRedirect[redirect_page] = []
        dictPageRedirect[redirect_page].append(Page(id,revid,title,meaningPage,multiPage,categories,links,redirect,first_sentense))
        dictRedirect[redirect_page].append(title)
        redirectcount +=1 
    pages.append(Page(id,revid,title,meaningPage,multiPage,categories,links,redirect,first_sentense))
    # json.dump(Page(id,revid,title,meaningPage,multiPage,categories,links,redirect),file,cls=PageEncoder,ensure_ascii=False)
    # file.write("\n")
#     if title == "Abbath" or title == "Аббате":
#         print(text, file = log)


4432391it [45:31, 1622.56it/s] 


In [5]:
# сохранение  информации о страницах
file = open("D:\\lbase_data\\ctxw.txt", "wb")
pickle.dump(pages, file=file)
file.close()
file = open("D:\\lbase_data\\drp.txt", "wb")
pickle.dump(dictPageRedirect, file=file)
file.close()
file = open("D:\\lbase_data\\dr.txt", "wb")
pickle.dump(dictRedirect, file=file)
file.close()
input.close()

In [4]:
file = open("D:\\lbase_data\\ctxw.txt", "rb")
unpickler = pickle.Unpickler(file)
pages = unpickler.load()
file.close()
file = open("D:\\lbase_data\\dr.txt", "rb")
unpickler = pickle.Unpickler(file)
dictRedirect = unpickler.load()
file.close()
file = open("D:\\lbase_data\\drp.txt", "rb")
unpickler = pickle.Unpickler(file)
dictPageRedirect = unpickler.load()
file.close()

In [5]:
for i, elem in enumerate(pages):
    if elem.title == "Компьютерный язык":
        print(i)
        break

1340


In [11]:
pages[1340].links


['язык',
 'компьютерная техника',
 'язык программирования',
 'программист',
 'компьютер',
 'Компьютерный сленг',
 'Язык программирования',
 'Сценарный язык',
 'Псевдоестественный язык',
 'Информационный язык',
 'Язык разметки',
 'Язык спецификаций',
 'Каскадные таблицы стилей',
 'Язык описания аппаратуры',
 'Протокол обмена',
 'сетевой протокол',
 'Компьютерный сленг']

In [6]:
@dataclass
class WikiSynset():
    def __init__(self, page:Page):
        self.page = page
        self.synset = [page]

    def append(self, redirect_title:Page):
        self.synset.append(redirect_title)


def includeTitleInWn(all_senses, title):
    title = title.upper()
    title = title.replace("—", "-")
    title = title.replace(",", "")
    if title in all_senses:
        return True
    if "(" in title:
        text = my_split(title).split(",")
        if text[0] in all_senses:
            return True
    text = my_split(title).split(",")
    lemmatized = " ".join([get_normal_form(word)
                for word in text[0].split()])
    if lemmatized.upper() in all_senses:
        return True
    if "Ё" in title:
        return includeTitleInWn(all_senses, title.replace("Ё","Е"))
    return False

In [7]:
print(len(dictPageRedirect))
print(len(dictRedirect))
    

988017
988017


In [8]:
wiki = []
meaningPageCounter = 0
multiPageCounter = 0
includeWS = 0
includeTitle = 0
wn = RuWordNet(filename_or_session='D:\\lbase_data\\ruwordnet.db')
all_senses = set([' '.join([get_normal_form(w).upper() for w in s.lemma.split()]) for s in wn.senses])
hashDict = {}
for index in tqdm(range(len(pages)-1)):
    hashDict[pages[index].title.lower()] = index
#пройтись по всем значения со страницы-значения и всем значения поставить мульти
i = 0
for index in tqdm(range(len(pages)-1)):
    if pages[index].meaningPage:
        for link in pages[index].links:
            if link.lower() in hashDict:
                pages[hashDict[link.lower()]].multiPage = True
                i += 1
print(i)
for page in tqdm(pages):
    if page.redirect:
        if includeTitleInWn(all_senses, page.title):
            includeTitle += 1
        continue
    wikiSyn = WikiSynset(page)
    if page.title in dictPageRedirect:
        for redirect in dictPageRedirect[page.title]:
            wikiSyn.append(redirect)
    if page.meaningPage:
        meaningPageCounter += 1
    if page.multiPage:
        multiPageCounter += 1
    wiki.append(wikiSyn)
    if includeTitleInWn(all_senses, page.title):
        includeTitle += 1
    # else:
    #     print(page.title)
page = 0
print(multiPageCounter)

100%|███████████████████████████████████████████████████████████████████| 4432390/4432390 [00:01<00:00, 2974754.09it/s]


407209


100%|█████████████████████████████████████████████████████████████████████| 4432391/4432391 [04:46<00:00, 15477.93it/s]

585956


In [8]:
# includeWS = 0
# for wikisyn in tqdm(wiki):
#     if includeTitleInWn(all_senses, str(wikisyn.page.title)) and (not wikisyn.page.multiPage) and (not wikisyn.page.meaningPage):
#             includeWS += 1
#     else:
#         for synset in wikisyn.synset:
#             if (includeTitleInWn(all_senses, str(synset.title)))and(not synset.multiPage) and (not synset.meaningPage):
#                 includeWS += 1
#                 break
# print("count title:", i)
# print("count redirect", redirectcount)
# print("meaningPage count", meaningPageCounter)
# print("multiPage count", multiPageCounter)
# print("WikiSynset count", len(wiki))
# print("include title in wn count", includeTitle)
# print("include solo wikisyn in wn count", includeWS)

In [9]:
@dataclass
class display:
    id:int
    revid:int
    title:str
    lemma:str
    wordId:int
    ctxW:set
    first_sentense:str
class displayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, display):
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)

In [10]:
def extractCtxW(links, categories):
    ctx = set()
    for link in links:
        ctx.add(" ".join([get_normal_form(word) for word in link.split()]))
    for elem in categories:
        ctx.add(" ".join([get_normal_form(word) for word in elem.split()]))
    return ctx

In [11]:
def unambiguousDisplay(title):
    title = title.upper()
    title = title.replace("—", "-")
    title = title.replace(",", "")
    if len(wn.get_synsets(title)) == 1:
        return [True, title]
    if "(" in title:
        text = my_split(title).split(",")
        if len(wn.get_synsets(text[0])) == 1:
            return [True, text[0]]
    text = my_split(title).split(",")
    lemmatized = " ".join([get_normal_form(word)
                for word in text[0].split()])
    if len(wn.get_synsets(lemmatized.upper())) == 1:
        return [True, lemmatized.upper()]
    if "Ё" in title:
        return unambiguousDisplay(title.replace("Ё","Е"))
    return [False, ""]

In [12]:
def extractCtxS(wn, lemma:str):
    #составляем контекст для слова из wordnet
    ctx_s = set()
    #synonymy
    for sense in wn.get_synsets(lemma):
        for synonymy in sense.senses:
            ctx_s.update(my_split(synonymy.lemma).split(","))
    #Hypernymy/Hyponymy
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            ctx_s.update(my_split(hypernyms.title).split(","))
    for sense in wn.get_senses(lemma):
        for hyponyms in sense.synset.hyponyms:
            ctx_s.update(my_split(hyponyms.title).split(","))
    #Sisterhood:
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            for sister in hypernyms.hyponyms:
                ctx_s.update(my_split(sister.title).split(","))
    return ctx_s
@dataclass
class WnCtx:
    id: int
    ctx: set
    lemmaInWn: str
    name: str
class WnCtxEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, WnCtx):
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)
def getKeyForId(key):
    if len(wn.get_senses(key)) > 0:
        # print(1)
        return wn.get_senses(key)[0].id
    if "(" in key:
        text = my_split(key).split(",")
        if  len(wn.get_senses(text[0])) > 0:
            # print(2)
            return wn.get_senses(text[0])[0].id
    text = my_split(key).split(",")
    lemmatized = " ".join([get_normal_form(word) for word in text[0].split()])
    if len(wn.get_senses(lemmatized)) > 0:
        # print(3)
        return wn.get_senses(lemmatized)[0].id
    if "ё" in key:
        return getKeyForId(key.replace("ё","у"))
    return ""

In [13]:
wn = RuWordNet(filename_or_session='D:\\lbase_data\\ruwordnet.db')
print(extractCtxS(wn, 'КОМПЬЮТЕРНЫЙ ЯЗЫК'))

{'ЯЗЫК ПРОГРАММИРОВАНИЕ', 'ИСКУССТВЕННЫЙ ЯЗЫК', 'ЭСПЕРАНТО', 'КОМПЬЮТЕРНЫЙ ЯЗЫК', 'ЯЗЫК ПРОГРАММИРОВАНИЯ'}


In [13]:
#read word in wn
mydoc = minidom.parse('D:\\lbase_data\\senses.N.xml')
items = mydoc.getElementsByTagName('sense')
countWn = 0
dictWn = {}
for elem in tqdm(items):
    countWn +=1
    text = elem.attributes['name'].value
    text_id = elem.attributes["id"].value
    lemma = elem.attributes["lemma"].value
    ctx_s = extractCtxS(wn, lemma)
    ctx = set()
    for elem in ctx_s:
        ctx.add(" ".join([get_normal_form(word) for word in elem.split()]))
    dictWn[text_id] = WnCtx(text_id, ctx, lemma, text)
print(len(dictWn), countWn)
print(wn.get_synsets("замок"))

  0%|                                                                              | 4/94710 [00:00<4:23:47,  5.98it/s]Error closing cursor
Traceback (most recent call last):
  File "C:\Users\Professional\anaconda3\lib\site-packages\sqlalchemy\engine\result.py", line 1696, in _fetchall_impl
    return list(self.iterator)
  File "C:\Users\Professional\anaconda3\lib\site-packages\sqlalchemy\orm\loading.py", line 147, in chunks
    fetch = cursor._raw_all_rows()
  File "C:\Users\Professional\anaconda3\lib\site-packages\sqlalchemy\engine\result.py", line 392, in _raw_all_rows
    rows = self._fetchall_impl()
  File "C:\Users\Professional\anaconda3\lib\site-packages\sqlalchemy\engine\cursor.py", line 1804, in _fetchall_impl
    return self.cursor_strategy.fetchall(self, self.cursor)
  File "C:\Users\Professional\anaconda3\lib\site-packages\sqlalchemy\engine\cursor.py", line 981, in fetchall
    self.handle_exception(result, dbapi_cursor, e)
  File "C:\Users\Professional\anaconda3\lib\site-p

KeyboardInterrupt: 

In [13]:
# file = open("D:\\lbase_data\\ctxS.txt", "wb")
# pickle.dump(dictWn, file=file)
# file.close()

In [14]:
file = open("D:\\lbase_data\\ctxS.txt", "rb")
unpickler = pickle.Unpickler(file)
dictWn = unpickler.load()
file.close()

In [15]:
# print(dictRedirect["Аббат"])
print(dictWn["130542-N-196389"])

WnCtx(id='130542-N-196389', ctx={'корректурный правка', 'тщательно проверить', 'выверить', 'вычитка текст', 'редактирование текст', 'модерация контент', 'вычитывание', 'вычитка'}, lemmaInWn='ВЫЧИТКА', name='ВЫЧИТКА')


In [16]:
def getLemmaForSynsets(title):
    title = title.upper()
    title = title.replace("—", "-")
    title = title.replace(",", "")
    ch = " " 
    if len(wn.get_synsets(title))>0:
        return title
    if "(" in title:
        text = my_split(title).split(",")
        if len(wn.get_synsets(text[0]))>0:
            if text[0][len(text[0]) - 1] == " ":
                text[0] = text[0].rstrip(ch)
            return text[0]
    text = my_split(title).split(",")
    lemmatized = " ".join([get_normal_form(word) for word in text[0].split()])
    if len(wn.get_synsets(lemmatized))>0:
        if lemmatized[len(lemmatized) - 1] == " ":
            lemmatized = lemmatized.rstrip(ch)
        return lemmatized
    if "Ё" in title:
        return getLemmaForSynsets(title.replace("Ё","Е"))
    return ""


In [17]:
dictDisplay = {} # словарь отображений
new_wiki = [] #будут все викисинсеты, кроме однозначных
wn = RuWordNet(filename_or_session='D:\\lbase_data\\ruwordnet.db')
for wikisyn in tqdm(wiki):
    if len(wikisyn.synset) != 1:
        new_wiki.append(wikisyn)
        continue
    one = unambiguousDisplay(wikisyn.page.title)
    if  (not wikisyn.page.meaningPage) and (not wikisyn.page.multiPage) and (getKeyForId(wikisyn.page.title) in dictWn) and one[0] :
        #id
        temp = wn.get_synsets(one[1])[0].id
        dictDisplay[wikisyn.page.title] = (display(wikisyn.page.id,wikisyn.page.revid,wikisyn.page.title,one[1], temp,
                                                  extractCtxW(wikisyn.page.links, wikisyn.page.categories), wikisyn.page.first_sentence))
    else:
        new_wiki.append(wikisyn)



100%|██████████████████████████████████████████████████████████████████████| 1826284/1826284 [12:34<00:00, 2420.66it/s]


In [18]:
# сохранение dictDisplay  и new_wiki
file = open("D:\\lbase_data\\dictDisplay1.txt", "wb")
pickle.dump(dictDisplay, file=file)
file.close()
file = open("D:\\lbase_data\\new_wiki.txt", "wb")
pickle.dump(new_wiki, file=file)
file.close()

In [14]:
file = open("D:\\lbase_data\\dictDisplay1.txt", "rb")
unpickler = pickle.Unpickler(file)
dictDisplay = unpickler.load()
file.close()
file = open("D:\\lbase_data\\new_wiki.txt", "rb")
unpickler = pickle.Unpickler(file)
new_wiki = unpickler.load()
file.close()

In [19]:
#     запись резулаов первого этапа в удобночиаемый вид
file = open("D:\\lbase_data\\unambiguousDisplay.txt", "w")
for key in dictDisplay:
    temp = "id:  "+str(dictDisplay[key].id) + " revid: " + str(dictDisplay[key].revid) + " title: "+str(dictDisplay[key].title)  + " lemma: "  + str(dictDisplay[key].lemma) + " wordId:" + str(dictDisplay[key].wordId)+"\n"
    file.write(temp)
file.close()

In [20]:
import gc
gc.collect()

155

In [21]:
#чуть раньше собираем кандидатов, для более точной
#проверки многозначности страниц
dictLemmaInIndex = {}
for index in tqdm(range(len(new_wiki))):
    lemma = getLemmaForSynsets(new_wiki[index].page.title)
    if lemma != "":
        if not lemma in dictLemmaInIndex:
            dictLemmaInIndex[lemma] = []
        dictLemmaInIndex[lemma].append(index)
    else:
        for elem in new_wiki[index].synset:
            lemma1 = getLemmaForSynsets(elem.title)
            if lemma1 != "":
                if not lemma1 in dictLemmaInIndex:
                    dictLemmaInIndex[lemma1] = []
                dictLemmaInIndex[lemma1].append(index)

100%|███████████████████████████████████████████████████████████████████████| 1823875/1823875 [48:59<00:00, 620.55it/s]


In [22]:
# сохранение dictLemmaInIndex
file = open("D:\\lbase_data\\dictLemmaInIndex.txt", "wb")
pickle.dump(dictLemmaInIndex, file=file)
file.close()

In [16]:
file = open("D:\\lbase_data\\dictLemmaInIndex.txt", "rb")
unpickler = pickle.Unpickler(file)
dictLemmaInIndex = unpickler.load()
file.close()

In [23]:
#дополнительная проверка на многозначность
dictTitleInIndex = {}
for index in tqdm(range(len(new_wiki))):
    dictTitleInIndex[new_wiki[index].page.title] = index
tempCount = 0
for key in dictLemmaInIndex:
    for elem in dictLemmaInIndex[key]:
        if not new_wiki[elem].page.multiPage:
            if len(dictLemmaInIndex[key])>1:
                new_wiki[dictTitleInIndex[new_wiki[elem].page.title]].page.multiPage = True
                tempCount +=1
print(tempCount)

100%|███████████████████████████████████████████████████████████████████| 1823875/1823875 [00:01<00:00, 1535248.67it/s]


16957


In [24]:
def helpForCheck(d, title):
    d = d.upper()
    d = d.replace("—", "-")
    d = d.replace(",", "")
    if d == title:
        return True
    if "(" in d:
        text = my_split(d).split(",")
        if title == text[0]:
            return True
    text = my_split(d).split(",")
    lemmatized = " ".join([get_normal_form(word) for word in text[0].split()])
    if lemmatized == title:
        return True
    if "Ё" in d:
        return helpForCheck(d.replace("Ё","Е"), d)
    return False
def check(wn, lemma, d):
    for sense in wn.get_synsets(lemma):
        if helpForCheck(d, sense.title):
            return [True, sense]
    return [False, "ф"]
print(len(new_wiki))
print(len(dictDisplay))
wiki3 = [] # будут только многозначные
# for w in tqdm(new_wiki):
#     if w.page.title == "Плебеи":
#         print(w.page.title, w.page.multiPage)
#         print(unambiguousDisplay(w.page.title))
#         print(getKeyForId(w.page.title))
#         break
countN = 0
for w in tqdm(new_wiki):
    flag = False
    if not w.page.meaningPage and not w.page.multiPage:
        for d in w.synset:
            one = unambiguousDisplay(d.title)
            #d.title in dictDisplay and
            if  one[0] and "N" in wn.get_synsets(one[1])[0].id:
                if w.page.title == "Abbath":
                    print(d.title)
                flag = True
                countN += 1
                idd = wn.get_synsets(one[1])[0].id
                p = display(w.page.id,w.page.revid,w.page.title,one[1], idd,
                            extractCtxW(w.page.links, w.page.categories), w.page.first_sentence)
                dictDisplay[w.page.title]=p
                break
    if not flag:
        wiki3.append(w)
print(len(dictDisplay), countN)

1823875
2409


100%|██████████████████████████████████████████████████████████████████████| 1823875/1823875 [24:56<00:00, 1218.68it/s]

7427 5018


In [25]:
file = open("D:\\lbase_data\\secondpart.txt", "w", encoding="utf-8")
for key in dictDisplay:
    temp = "id:  "+str(dictDisplay[key].id) + " revid: " + str(dictDisplay[key].revid) + " title: "+str(dictDisplay[key].title)  + " lemma: "  + str(dictDisplay[key].lemma) + " wordId:" + str(dictDisplay[key].wordId)+"\n"
    file.write(temp)
file.close()

In [26]:
# сохранение dictDisplay  и wiki3
file = open("D:\\lbase_data\\dictDisplay2.txt", "wb")
pickle.dump(dictDisplay, file=file)
file.close()
file = open("D:\\lbase_data\\wiki3.txt", "wb")
pickle.dump(wiki3, file=file)
file.close()

In [17]:
file = open("D:\\lbase_data\\dictDisplay2.txt", "rb")
unpickler = pickle.Unpickler(file)
dictDisplay = unpickler.load()
file.close()
file = open("D:\\lbase_data\\wiki3.txt", "rb")
unpickler = pickle.Unpickler(file)
wiki3 = unpickler.load()
file.close()

In [27]:
#теперь собираем кандидатов только из оставшихся статей
dictLemmaInIndex = {}
for index in tqdm(range(len(wiki3))):
    lemma = getLemmaForSynsets(wiki3[index].page.title)
    if lemma != "":
        if not lemma in dictLemmaInIndex:
            dictLemmaInIndex[lemma] = []
        dictLemmaInIndex[lemma].append(index)
    else:
        for elem in wiki3[index].synset:
            lemma1 = getLemmaForSynsets(elem.title)
            if lemma1 != "":
                if not lemma1 in dictLemmaInIndex:
                    dictLemmaInIndex[lemma1] = []
                dictLemmaInIndex[lemma1].append(index)

100%|███████████████████████████████████████████████████████████████████████| 1818857/1818857 [48:35<00:00, 623.89it/s]


In [28]:
# сохранение dictLemmaInIndex
file = open("D:\\lbase_data\\dictLemmaInIndex1.txt", "wb")
pickle.dump(dictLemmaInIndex, file=file)
file.close()

In [18]:
file = open("D:\\lbase_data\\dictLemmaInIndex1.txt", "rb")
unpickler = pickle.Unpickler(file)
dictLemmaInIndex = unpickler.load()
file.close()

In [29]:
#собираем кандидатов
from collections import defaultdict
dictSynsetId = defaultdict(list)
countLinksAdd = 0
wn = RuWordNet(filename_or_session='D:\\lbase_data\\ruwordnet.db')
for index in tqdm(range(len(wiki3))):
    w = wiki3[index]
    if not w.page.meaningPage:
        lemma = getLemmaForSynsets(w.page.title)
        #если нашли лемму из ворднета и она существительное
        if lemma != "" and getKeyForId(lemma) in dictWn:
            for synset in wn.get_synsets(lemma):
                dictSynsetId[synset.id].append(w)
    else:
        lemmaTitle = getLemmaForSynsets(w.page.title)
        # if lemmaTitle.lower() == "тихий океан":
        #     print(1111, index, w.page.links, w.page.title, w.page.meaningPage)
        for link in w.page.links:
            lemma = getLemmaForSynsets(link)
            if lemma in dictLemmaInIndex:
                #если нашли лемму многозначной статьи из ворднета и она существительное
                if lemmaTitle != "" and getKeyForId(lemmaTitle) in dictWn:
                    for synset in wn.get_synsets(lemmaTitle):
                        countLinksAdd += 1
                        if countLinksAdd < 5:
                            print(lemma,countLinksAdd)
                        for indexElem in dictLemmaInIndex[lemma]:
                            dictSynsetId[synset.id].append(wiki3[indexElem])
print(countLinksAdd)


  0%|                                                                           | 145/1818857 [00:00<42:02, 720.90it/s]

ВОЛГА 1
АЙОВА 2
АЙОВА 3
АЙОВА 4


100%|██████████████████████████████████████████████████████████████████████| 1818857/1818857 [22:16<00:00, 1360.55it/s]

48829


In [30]:
# сохранение dictSynsetId
file = open("D:\\lbase_data\\dictSynsetId.txt", "wb")
pickle.dump(dictSynsetId, file=file)
file.close()

In [19]:
file = open("D:\\lbase_data\\dictSynsetId.txt", "rb")
unpickler = pickle.Unpickler(file)
dictSynsetId = unpickler.load()
file.close()

In [31]:
#возможны повторения среди кандидатов, уадлим их
def count1(value, array):
    count = 0
    for elem in array:
        if elem.page.id == value.page.id:
            count += 1
    return count
for key in tqdm(dictSynsetId):
    tempList = []
    for elem in dictSynsetId[key]:
        if count1(elem, tempList) < 1 and not elem.page.meaningPage:
            tempList.append(elem)
    dictSynsetId[key] = tempList
#sort candidates
sorted_tuple = sorted(dictSynsetId.items(), key=lambda x: x[0])
sortCandidates = dict(sorted_tuple)
dictIdTitle = {}
for synset in wn.synsets:
    dictIdTitle[synset.id] = synset.title
#запись в файл претендентов
candidates = open("D:\\lbase_data\\candidates.txt", "w", encoding="utf-8")
i = 0
for key in tqdm(sortCandidates):
    s = "id synset = "+str(key) + " synset title = " + dictIdTitle[key]
    for elem in sortCandidates[key]:
        s += " title = " + elem.page.title +" id = " + elem.page.id + " "
    print(s, file = candidates)
candidates.close()

100%|█████████████████████████████████████████████████████████████████████████| 20502/20502 [00:00<00:00, 67663.00it/s]


In [32]:

def score(ctxS, ctxW):
    return len(ctxS.intersection(ctxW))
def get_key(key, dictWn):
    key = key.lower()
    key = key.replace("—", "-")
    key = key.replace(",", "")
    id = getKeyForId(key)
    if id in dictWn:
        # print(1)
        return id
    if "(" in key:
        id = wn.get_senses(text[0])[0].id
        text = my_split(key).split(",")
        if  id in dictWn:
            # print(2)
            return id
    text = my_split(key).split(",")
    lemmatized = " ".join([get_normal_form(word) for word in text[0].split()])
    id = wn.get_senses(lemmatized)[0].id
    if id in dictWn:
        # print(3)
        return id
    if "ё" in key:
        return get_key(key.replace("ё","у"), dictWn)
    return ""

In [33]:
labse = SentenceBertTransformer(device="cuda")
labse.load_model()

In [34]:
def cosin_distance(word, sentense):
    word_embeding = labse.transform(word)
    sentense_embeding = labse.transform(sentense)
    return np.dot(word_embeding, sentense_embeding) / (sum(sentense_embeding ** 2) * sum(word_embeding ** 2))

In [35]:
import random
#3 этап алгоритма
badlemma = []
baddenominator = []
badmaxP = []
badsynsetlemma = []
badidWn = []
print(len(dictDisplay))
dictSortCandidates = {}
for key in tqdm(sortCandidates):
    if len(sortCandidates[key]) == 1:
            w = sortCandidates[key][0]
            p = display(w.page.id,w.page.revid,w.page.title,dictIdTitle[key], key,extractCtxW(w.page.links, w.page.categories), w.page.first_sentence)
            dictDisplay[w.page.title]=p
            dictSortCandidates[key] = [(sortCandidates[key][0], 1)]
    else:
        maxP = -1
        maxagrument = 0
        lemmaSynset = getLemmaForSynsets(dictIdTitle[key])
        dictSortCandidates[key] = []
        if lemmaSynset != "":
            idWn = wn.get_senses(lemmaSynset)[0].id
            if "N" in idWn: #почему-то иногда для синсета существительного сенс не существительное
                for elem in sortCandidates[key]:
                    ctxw = extractCtxW(elem.page.links, elem.page.categories)
                    lemma = getLemmaForSynsets(elem.page.title)
                    if lemma != "":
                        numerator = score(dictWn[idWn].ctx, ctxw)
                        denominator = 0
                        for item in sortCandidates[key]:
                            addctxw = extractCtxW(item.page.links, item.page.categories)
                            denominator +=score(dictWn[idWn].ctx, addctxw)
                    else:
                        badlemma.append(elem.page.title)
                    temp_distance = cosin_distance(lemmaSynset, elem.page.first_sentence)
                    if denominator != 0:
                        temp_p = numerator / denominator +  temp_distance 
                        dictSortCandidates[key].append((elem, temp_p))                           
                        if  temp_p > maxP:
                            maxP = temp_p
                            maxagrument = elem
                    else:
                        if  temp_distance > maxP:
                            maxP = temp_distance
                            maxagrument = elem
                        baddenominator.append(elem.page.title)
                        dictSortCandidates[key].append((elem, temp_distance))
            else:
                badidWn.append(wn.get_senses(lemmaSynset)[0].id)
        else:
            badsynsetlemma.append(dictIdTitle[key])
        if maxP != - 1:
            w = maxagrument
            p = display(w.page.id,w.page.revid,w.page.title,dictIdTitle[key], key,
                        extractCtxW(w.page.links, w.page.categories), w.page.first_sentence)
            dictDisplay[w.page.title]=p
        else:
            badmaxP.append(key)
print("len(dictDisplay)",len(dictDisplay)) 
print("len(badlemma)",len(badlemma))
print("len(baddenominator)",len(baddenominator))
print("len(badmaxP)",len(badmaxP))
print("len(badsynsetlemma)",len(badsynsetlemma))
print("len(badidWn)",len(badidWn))

7427


100%|████████████████████████████████████████████████████████████████████████████| 20502/20502 [31:04<00:00, 10.99it/s]

len(dictDisplay) 22313
len(badlemma) 14130
len(baddenominator) 18086
len(badmaxP) 2415
len(badsynsetlemma) 1638
len(badidWn) 738


In [36]:
for key in dictSortCandidates:
    temp = sorted(dictSortCandidates[key],key=lambda x: x[1], reverse = True)
    dictSortCandidates[key] = temp
candidates = open("D:\\lbase_data\\sortcandidates.txt", "w", encoding="utf-8")
candidates1 = open("D:\\lbase_data\\sortcandidates1000.txt", "w", encoding="utf-8")
i = 0
for key in tqdm(dictSortCandidates):
    s = "id synset: "+str(key) + "\tsynset title: " + dictIdTitle[key] + "\n" 
    lemmaSynset = getLemmaForSynsets(dictIdTitle[key])
    if lemmaSynset != "" :
        idWn = wn.get_senses(lemmaSynset)[0].id
        if "N" in idWn:
            s += "Ctx synset: " + ",".join(list(dictWn[idWn].ctx)) + "\n"
        else:
            s += "id for N is A. lemma: " + lemmaSynset + "\n"
    else:
        s += "lemmasynset not found\n" 
    for elem in dictSortCandidates[key]:
        s += "title: " + elem[0].page.title +" id = " + elem[0].page.id + " " + str(elem[1]) + "\n"
        s += "ctx article:" +",".join(list(extractCtxW(elem[0].page.links,elem[0].page.categories))) +"\n"
    s += "\n"
    print(s, file = candidates)
    if i < 1000:
        print(s, file = candidates1) 
    i += 1
candidates.close()
candidates1.close()

100%|███████████████████████████████████████████████████████████████████████████| 20502/20502 [01:12<00:00, 281.19it/s]


In [37]:
# сохранение dictDisplay
file = open("D:\\lbase_data\\dictDisplay3.txt", "wb")
pickle.dump(dictDisplay, file=file)
file.close()

In [54]:
file = open("D:\\lbase_data\\dictDisplay3.txt", "rb")
unpickler = pickle.Unpickler(file)
dictDisplay = unpickler.load()
file.close()

In [38]:
file = open("D:\\lbase_data\\thirdpart.txt","w", encoding="utf-8")
for key in tqdm(dictDisplay):
    temp = "id:  "+str(dictDisplay[key].id) + " wordId: " + str(dictDisplay[key].wordId) + " title: "+str(dictDisplay[key].title) +" lemma: "+str(dictDisplay[key].lemma) + '\n'
    file.write(temp)
file.close()

100%|████████████████████████████████████████████████████████████████████████| 22313/22313 [00:00<00:00, 429099.71it/s]


In [55]:
#собираем отображения  связанные содним wordId
dict_wordId_in_display_and_key = defaultdict(list)
for key, value in dictDisplay.items():
    dict_wordId_in_display_and_key[value.wordId].append((key,value))
i  = 0
for key, value in dict_wordId_in_display_and_key.items():
    if len(value) > 1:
        i+=len(value)
i, len(dict_wordId_in_display_and_key), len(dictDisplay)

(4603, 19878, 22313)

In [56]:
#удалим из dictDisplay повторяющиеся wordId
for key, value in dict_wordId_in_display_and_key.items():
    if len(value) > 1:
        for item in value:
            del dictDisplay[item[0]]
#удалим из dict_wordId_in_display_and_key однозначные отображенния
dict_wordId_in_display_and_key_new = defaultdict(list)
for key, value in dict_wordId_in_display_and_key.items():
    if len(value) > 1:
        dict_wordId_in_display_and_key_new[key] = value
len(dict_wordId_in_display_and_key_new), len(dictDisplay)

(2168, 17710)

In [57]:
dict_wordId_in_display_and_key_new["7651-N"]

[('Компьютерный язык',
  display(id='2661', revid='2184205', title='Компьютерный язык', lemma='КОМПЬЮТЕРНЫЙ ЯЗЫК', wordId='7651-N', ctxW={'каскадный таблица стиль', 'язык разметка', 'компьютер', 'язык описание аппаратура', 'компьютерный техника', 'язык спецификация', 'протокол обмен', 'информационный язык', 'компьютерный сленг', 'компьютерный язык', 'язык', 'псевдоестественный язык', 'сетевой протокол', 'сценарный язык', 'язык программирование', 'программист'}, first_sentense='Понятие компью́терный язы́к (калька с ), как правило, относится к языкам, ассоциируемым с компьютерной техникой')),
 ('Язык программирования',
  display(id='2728', revid='121215610', title='Язык программирования', lemma='ЯЗЫК ПРОГРАММИРОВАНИЯ', wordId='7651-N', ctxW={'симула', 'логика первый порядок', 'системный программирование', 'зависимый тип', 'компьютер', 'приведение тип', 'планкалкюль', 'тест производительность', 'утечка память', 'стандарт оформление код', 'мемоизация', 'хоар', 'структурный программирование

In [58]:
#проведем похожий третьему этапу обор кандидатов
dictIdTitle = {}
i = 0
for synset in wn.synsets:
    dictIdTitle[synset.id] = synset.title
for key, value in tqdm(dict_wordId_in_display_and_key_new.items()):
    lemmaSynset = getLemmaForSynsets(wn[str(key)].title.lower())
    if lemmaSynset:
        idWn = wn.get_senses(lemmaSynset)[0].id 
        if "N" in idWn:
            ctxS = dictWn[idWn].ctx
            maxP = -1
            arg_max = 0
            for item in value:
                numerator = score(ctxS, item[1].ctxW)
                deniminator = 0.0
                for num in value:
                    deniminator += score(ctxS, num[1].ctxW)
                p = cosin_distance(lemmaSynset, item[1].first_sentense)
                if deniminator != 0.0:
                    p += numerator/deniminator
                if p > maxP:
                    maxP = p
                    arg_max = item
            if maxP != -1:
                dictDisplay[arg_max[0]] = arg_max[1]
            else:
                i+=1
#                 print(f"denominator = 0 for all candidates for key:{key} synset is {wn[str(key)].title}")
        else:
            i+=1
#             print(f"lemma for key {key} is not noun, lemmaSynset is {lemmaSynset} for sysnet {wn[str(key)].title}")        
    else:
        i+=1
#         print(f"lemmaSynset for key {key} is none, synset is {wn[str(key)].title}")

100%|██████████████████████████████████████████████████████████████████████████████| 2168/2168 [00:51<00:00, 42.31it/s]


In [59]:
len(dictDisplay), i

(19775, 103)

In [60]:
# сохранение dictDisplay
file = open("D:\\lbase_data\\dictDisplay4.txt", "wb")
pickle.dump(dictDisplay, file=file)
file.close()

In [ ]:
file = open("D:\\lbase_data\\dictDisplay4.txt", "rb")
unpickler = pickle.Unpickler(file)
dictDisplay = unpickler.load()
file.close()

In [61]:
len(dictDisplay)

19775

In [62]:
file = open("D:\\lbase_data\\fourthpart.txt","w", encoding="utf-8")
for key in tqdm(dictDisplay):
    temp = "id:  "+str(dictDisplay[key].id) + " wordId: " + str(dictDisplay[key].wordId) + " title: "+str(dictDisplay[key].title) +" lemma: "+str(dictDisplay[key].lemma) + '\n'
    file.write(temp)
file.close()

100%|████████████████████████████████████████████████████████████████████████| 19775/19775 [00:00<00:00, 520406.33it/s]


In [63]:
import pandas as pd
df = pd.read_csv("D:\\lbase_data\\wiki_merge_predict_test_29_11_22.csv")
df

,Unnamed: 0,synset_id,wiki_title,annotation,wiki_title_gold
0,0,115195-N,Облава,Да,Облава
1,1,128158-N,Шаньдун,Да,Шаньдун
2,2,143699-N,Оксюморон,Да,Оксюморон
3,3,4061-N,Доверенность,Да,Доверенность
4,5,136544-N,Шахматная доска,Да,Шахматная доска
...,...,...,...,...,...
1725,1993,113701-N,Царю небесный,Нет,Бог
1726,1994,124807-N,Наставничество,Да,Наставничество
1727,1995,9199-N,Трансурановые элементы,Да,Трансурановые элементы
1728,1996,103410-N,Эсватини,Да,Эсватини


In [64]:
synset_id = set(df["synset_id"].values)

In [65]:
dict_for_check = {}
for key, value in tqdm(dictDisplay.items()):
    if value.wordId in synset_id:
        dict_for_check[value.wordId] = value.title

100%|███████████████████████████████████████████████████████████████████████| 19775/19775 [00:00<00:00, 1977454.74it/s]


In [66]:
dataset = df[["synset_id", "wiki_title_gold"]]
for_apply = df["synset_id"]
dataset["wiki_title"] = for_apply.apply(lambda x: dict_for_check[x] if x in dict_for_check else "не связан")
dataset

C:\Users\Professional\AppData\Local\Temp\ipykernel_2312\3563258175.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["wiki_title"] = for_apply.apply(lambda x: dict_for_check[x] if x in dict_for_check else "не связан")


,synset_id,wiki_title_gold,wiki_title
0,115195-N,Облава,Облава
1,128158-N,Шаньдун,Шаньдун
2,143699-N,Оксюморон,Оксюморон
3,4061-N,Доверенность,Доверенность
4,136544-N,Шахматная доска,не связан
...,...,...,...
1725,113701-N,Бог,Бог
1726,124807-N,Наставничество,Наставничество
1727,9199-N,Трансурановые элементы,Трансурановые элементы
1728,103410-N,Эсватини,Эсватини


In [67]:
from sklearn.metrics import accuracy_score
y_pred = dataset["wiki_title"].values
y_true = dataset["wiki_title_gold"].values
accuracy_score(y_true, y_pred)

0.7416184971098266